In [ ]:
!nvidia-smi

Thu Jan  7 03:28:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('data/')

Mounted at data/


In [ ]:
import pandas as pd
df = pd.read_csv('/content/data/MyDrive/Cassava Leaf/train.csv')

k = 0
for i in range(21397):
    try:
        if df.loc[i]['label'] == 3:
            if k<10581:
                df = df.drop(i, axis= 0)
                k+=1
            else:
                break
    except:
        pass

images = list(df['image_id'])[:-1]
labels= list(df['label'])[:-1]

len(images), len(labels)

(10815, 10815)

In [ ]:
df.label.value_counts()

4    2577
3    2577
2    2386
1    2189
0    1087
Name: label, dtype: int64

In [ ]:
!pip install efficientnet_pytorch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch
import h5py
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import datetime
import matplotlib.pyplot as plt
from torchsummary import summary
from efficientnet_pytorch import EfficientNet
import os
from multiprocessing import Process, Pool

from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16032 sha256=a678b77c951c34777109d9eef5b968d3e53e52eac28855d5f581660c8a2aa911
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, path, images, labels, transform):
        self.path = path
        self.images = images
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):

        label = pd.get_dummies(self.labels).loc[idx]
        img = Image.open(self.path+self.images[idx])
        img = self.transform(img)
        return (img.to('cuda'), torch.tensor(label).to('cuda'))


    def __len__(self):
        return len(self.images)

In [ ]:
path = r'/content/data/MyDrive/Cassava Leaf/train_images/'

transform = transforms.Compose([
                transforms.Resize((400, 600)), # 600x800
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation((-30, 30))
])

dataset = CassavaDataset(path, images, labels, transform)

In [ ]:
len(dataset)

10815

In [ ]:
trainset, testset = torch.utils.data.random_split(dataset, [9999, 816])
testset, valset = torch.utils.data.random_split(testset, [408, 408])

train_dl = DataLoader(trainset, batch_size= 16, shuffle= True)
test_dl = DataLoader(testset, batch_size= 16, shuffle= True)
val_dl = DataLoader(valset, batch_size= 16, shuffle= True)

In [ ]:
effnet = EfficientNet.from_name('efficientnet-b3') #b1 and b2
effnet._fc = nn.Linear(in_features=1536, out_features=5, bias=True)
effnet = torch.load(r'/content/data/MyDrive/cassava_effnet_b3_20_lesser')
effnet = effnet.to('cuda')
summary(effnet, (3, 600, 800))
# effnet

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 602, 802]               0
Conv2dStaticSamePadding-2         [-1, 40, 300, 400]           1,080
       BatchNorm2d-3         [-1, 40, 300, 400]              80
MemoryEfficientSwish-4         [-1, 40, 300, 400]               0
         ZeroPad2d-5         [-1, 40, 302, 402]               0
Conv2dStaticSamePadding-6         [-1, 40, 300, 400]             360
       BatchNorm2d-7         [-1, 40, 300, 400]              80
MemoryEfficientSwish-8         [-1, 40, 300, 400]               0
          Identity-9             [-1, 40, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 10, 1, 1]             410
MemoryEfficientSwish-11             [-1, 10, 1, 1]               0
         Identity-12             [-1, 10, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 40, 1, 1]             440
         I

In [ ]:
len(trainset)

9999

In [ ]:
len(train_dl)

625

In [ ]:
loss_fun = nn.MSELoss()

def encode(out):
    for i in range(len(out)):
        for j in range(len(out[i])):
            if out[i][j] == torch.max(out[i]):
                out[i][j] = 1
            else:
                out[i][j] = 0
    return out

def accuracy(out, labels):
    c=0
    output = out.clone()
    label = labels.clone()
    outa = encode(output)
    labelsa = encode(label)
    for i in range(len(outa)):
        if list(outa[i]==labelsa[i]) == [True, True, True, True, True]:
            c+=1
    return c/len(output)

def evaluate(model, val_dl):
    model.eval()
    val_acc, val_loss = [], []
    for batch in val_dl:
        img, labels = batch
        with torch.no_grad():
            out = model(img)
            loss = loss_fun(out, labels.long())
            acc = accuracy(out, labels)
            val_loss.append(loss.detach())
            val_acc.append(acc)
    return sum(val_loss)/len(val_loss), sum(val_acc)/len(val_acc)

def fit(model, epochs, train_dl, validation_dl, lr=1e-3):
    model.train()
    optimizer= torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    train_acc, train_loss, val_acc, val_loss= [], [], [], []
    for epoch in range(epochs):
        epoch_loss, epoch_acc = [], []
        for batch in train_dl:
            model.train()
            img, labels = batch
            out = model(img)
            loss = loss_fun(out, labels.float())
            acc = accuracy(out, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss.append(loss)
            epoch_acc.append(acc)
            
        train_loss.append(sum(epoch_loss)/len(epoch_loss))
        train_acc.append(sum(epoch_acc)/len(epoch_acc))
        vald_loss, vald_acc  = evaluate(model, val_dl)
        val_acc.append(vald_acc)
        val_loss.append(vald_loss)
        print(f'Epoch: {epoch+1}/{epochs} -- Train Loss: {sum(epoch_loss)/len(epoch_loss)} -- Train Acc: {sum(epoch_acc)*100/len(epoch_acc)} -- Val Loss: {vald_loss} -- Val Acc: {vald_acc*100}') 

        torch.save(effnet, '/content/data/MyDrive/cassava_effnet_b3_20_lesser')
    return {"train_loss": train_loss , "train_acc" : train_acc, "val_loss" : val_loss, "val_acc": val_acc}

In [ ]:
history = fit(effnet, 10, train_dl, val_dl, lr=1e-3)

Epoch: 1/10 -- Train Loss: 0.08491052687168121 -- Train Acc: 70.12733333333334 -- Val Loss: 0.08591455966234207 -- Val Acc: 71.15384615384616
Epoch: 2/10 -- Train Loss: 0.08346998691558838 -- Train Acc: 71.29533333333333 -- Val Loss: 0.08151596784591675 -- Val Acc: 71.39423076923077


In [ ]:
evaluate(effnet, test_dl)

In [ ]:
torch.save(effnet, '/content/data/MyDrive/cassava_effnet_b3_20_lesser')